In [1]:
%matplotlib widget
import matplotlib.pyplot as plt

In [2]:
import ee
service_account = 'climate-hazard-demo@data-portal-adaptation.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, '../dataportal/data-portal-adaptation.json')

In [3]:
# GEE authentication using service account key stored in local directory
ee.Initialize(credentials)

In [4]:
import geemap
from ipyleaflet import CircleMarker, Polygon, LayersControl, DrawControl, ZoomControl
from ipywidgets import interact, interactive, interact_manual, Layout, Accordion
import ipywidgets as widgets
import sys, asyncio, codecs, json
import numpy as np
from midiutil import MIDIFile
from music21 import midi

In [5]:
# Create decorator to prevent events triggers while slider is being moved
# From https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Events.html
class Timer:
    def __init__(self, timeout, callback):
        self._timeout = timeout
        self._callback = callback

    async def _job(self):
        await asyncio.sleep(self._timeout)
        self._callback()

    def start(self):
        self._task = asyncio.ensure_future(self._job())

    def cancel(self):
        self._task.cancel()

def debounce(wait):
    """ Decorator that will postpone a function's
        execution until after `wait` seconds
        have elapsed since the last time it was invoked. """
    def decorator(fn):
        timer = None
        def debounced(*args, **kwargs):
            nonlocal timer
            def call_it():
                fn(*args, **kwargs)
            if timer is not None:
                timer.cancel()
            timer = Timer(wait, call_it)
            timer.start()
        return debounced
    return decorator

In [6]:
# HAZARD INFO

hazards = {
    'maxtemp': {
         'keyname': 'maxtemp',
        'short_displayname': 'mxT',
         'displayname': 'maximum annual temperature',
        'unit': 'C',
         'min': -10,
         'max': 60,
         'use_greaterthan': True,
         'years': list(range(2020, 2100)),
         'num_bins': 70,
         'bin_width': 1,
         'hazard_category': 'heat',
        'narrative_phrase': 'temperatures as high as {:.0f} C'
    },
    'mintemp': {
         'keyname': 'mintemp',
        'short_displayname': 'mnT',
         'displayname': 'minimum annual temperature',
        'unit': 'C',
         'min': -110,
         'max': 46,
         'use_greaterthan': False,
         'years': list(range(2020, 2100)),
         'num_bins': 78,
         'bin_width': 2,
         'hazard_category': 'cold',
        'narrative_phrase': 'temperatures as low as {:.0f} C'
    },
    'maxprecip': {
        'keyname': 'maxprecip',
        'short_displayname': 'mxPR',
         'displayname': 'maximum daily precipitation',
        'unit': 'mm',
         'min': 0,
         'max': 4000,
         'use_greaterthan': True,
         'years': list(range(2020, 2100)),
         'num_bins': 80,
         'bin_width': 20,
         'hazard_category': 'precipitation',
        'narrative_phrase': 'as many as {:.0f} mm precipitation in a day'
    },
    'maxdryspell': {
        'keyname': 'maxdryspell',
        'short_displayname': 'mxDS',
        'displayname': 'max dryspell duration',
        'unit': 'days',
        'min': 0,
        'max': 365,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 73,
        'bin_width': 5,
        'hazard_category': 'drought',
        'narrative_phrase': 'dryspells of up to {:.0f} days'
    },
    'mtt35': {
        'keyname': 'mtt35',
        'short_displayname': 'MTT35',
        'displayname': 'days temp above 35',
        'unit': 'days',
        'min': 0,
        'max': 300,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 74,
        'bin_width': 5,
        'hazard_category': 'heat',
        'narrative_phrase': "as many as {:.0f} days above 35 C"
    },
    'modlandslide': {
        'keyname': 'modlandslide',
        'short_displayname': 'LS',
        'displayname': 'days moderate landslide risk',
        'unit': 'days',
        'min': 0,
        'max': 300,
        'use_greaterthan': True,
        'years': list(range(2020, 2100)),
        'num_bins': 60,
        'bin_width': 5,
        'hazard_category': 'landslide',
        'narrative_phrase': 'as many as {:.0f} days of moderate-to-high landslide risk'
    }
}

hazard_data = {hazard: {
    'probs': ee.ImageCollection('users/tedwongwri/dataportal/posterior_probs/{}'.format(hazard)),
    'tprobs': ee.ImageCollection('users/tedwongwri/dataportal/thresholdposterior_probs/{}'.format(hazard)),
    }
    for hazard in hazards.keys()
}

In [7]:
def ev(year, hazard):
# Return expected value of hazard for given year
    probs = hazard_data[hazard]['probs'].filterMetadata('year', 'equals', year).first().reduceRegion(reducer=ee.Reducer.mean(), geometry=current_geom, scale=250, bestEffort=True).getInfo()
    if np.sum([float(i) for i in probs.values()]) == 0:
        return 0, 1
    dataprobs = np.array([float(i) for i in probs.values()]) / np.sum([float(i) for i in probs.values()])
    mag_range = [(j.split('-')[0], j.split('-')[1]) for j in [i.split('_')[1] for i in list(probs.keys())]]
    avg_mags = np.array([(float(i[0].replace('neg', '-')) + float(i[1].replace('neg', '-'))) / 2 for i in mag_range])
    ev = np.dot(dataprobs, avg_mags) 
    return ev

In [8]:
# DATA AND INITIAL SETTINGS

initial_threshold = 30
min_threshold = 5
max_threshold = 50
threshold_interval = 5
DEFAULT_CURRENTYEAR = 2022
currentYear = DEFAULT_CURRENTYEAR
futureYear = 2050
prev_threshold_current = initial_threshold
prev_threshold_future = initial_threshold
prev_futureYear = 2050

prev_layers = []

persistent_threshold_value = initial_threshold
persistent_ev_current = None
persistent_ev_future = None
persistent_threshold_value = initial_threshold
#initial_coords = [50.85045, 4.34878]  # Brussels
initial_coords = [-70.575, -33.383333] # Vitacura  -- Give as lat, lon
current_geom = ee.Geometry.Point(initial_coords)
current_location_marker = None
ev_val = 'expected'
mode_is_ev = False

auto_thresholds = True
percentilebased_lookup_thresholds = {
    hn: hazards[hn]['min'] for hn in hazards.keys()
}
hazard_threshold_sliders = {}
userdefined_lookup_thresholds = {
    hn: hazards[hn]['min'] for hn in hazards.keys()
}
highrisk_categories = {'current': [], 'future': []}
user_ever_defined = False

current_hazard = 'maxtemp'

MAPprobs = hazard_data[current_hazard]['tprobs']
currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()

def update_data():
    global MAPprobs
    global currentprobs
    global futureprobs
    MAPprobs = hazard_data[current_hazard]['tprobs']
    currentprobs = MAPprobs.filterMetadata('year', 'equals', currentYear).first()
    futureprobs = MAPprobs.filterMetadata('year', 'equals', futureYear).first()

In [35]:
# SOUNDS

def make_midi(): 
    update_status('Processing high temps')
    hightemps = [ev(year, 'maxtemp') for year in range(2022, 2100)]
    update_status('Processing low temps')
    lowtemps = [ev(year, 'mintemp') for year in range(2022, 2100)]
    update_status('Processing precip')
    maxprecips = [ev(year, 'maxprecip') for year in range(2022, 2100)]
    update_status('Creating sound file')
    highpiano_degrees  = [max(round((temp/50)*127), 0) for temp in hightemps]  # MIDI note number 0-127
    lowpiano_degrees  = [max(round(temp + 30), 0) for temp in lowtemps]
    cello_degrees = [max(round((precip - 300)*0.7), 0) for precip in maxprecips]


    track    = 0
    channel  = 0
    time     = 0    # In beats
    duration = 1    # In beats
    tempo    = 240   # In BPM
    volume   = 80  # 0-127, as per the MIDI standard

    outMIDI = MIDIFile(2)  # One track, defaults to format 1 (tempo track is created
                      # automatically)
    outMIDI.addTempo(track, time, tempo)
    outMIDI.addTimeSignature(track, time, 4, 2, 24, 8)
    outMIDI.addProgramChange(0, 0, 0, 0)
    outMIDI.addProgramChange(1, 1, 0, 45)

    track = 0
    channel = 0
    for i, pitch in enumerate(highpiano_degrees):
        outMIDI.addNote(track, channel, pitch, time + i, duration, volume)



    running_duration = 1
    pitch_first_time = 0
    track = 0
    channel = 0
    for i in range(len(lowpiano_degrees) - 1):
        pitch = lowpiano_degrees[i]
        next_pitch = lowpiano_degrees[i+1]
        if (pitch != next_pitch) or (i % 4 == 3):
            outMIDI.addNote(track, channel, pitch + 35, pitch_first_time, running_duration, round(volume*1))
            running_duration = 1
            pitch_first_time = i + 1
        else:
            running_duration += 1

    running_duration = 1
    pitch_first_time = 0
    track = 1
    channel = 1
    for i in range(len(cello_degrees) - 1):
        pitch = cello_degrees[i]
        next_pitch = cello_degrees[i+1]
        if (pitch != next_pitch) or (i % 4 == 3):
            outMIDI.addNote(track, channel, pitch+5, pitch_first_time, running_duration, round(volume*1))
            running_duration = 1
            pitch_first_time = i + 1
        else:
            running_duration += 1

    with open("midi_out.mid", "wb") as output_file:
        outMIDI.writeFile(output_file)
    update_status('Ready')

In [10]:
def playMidi(filename):
    mf = midi.MidiFile()
    mf.open(filename)
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show('midi')

In [11]:
def showMidi(filename):
    mf = midi.MidiFile()
    mf.open(filename)
    mf.read()
    mf.close()
    s = midi.translate.midiFileToStream(mf)
    s.show()

In [12]:
def make_music():
    make_midi()
    playMidi('midi_out.mid')

In [13]:
# MAP

def latlon_to_lonlat(coords):
# Converts EE coords (x, y) to ipyleaflet coords (y, x)
# Works for geojson coords
    if type(coords[0]) == float:
        return [coords[1], coords[0]]
    else:
        return [[[i[1],i[0]] for i in j] for j in coords]

upload_button = widgets.FileUpload(
    accept = '.geojson',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple = False,  # True to accept multiple files upload else False
    description = 'GeoJSON'
)

Map = geemap.Map(center=latlon_to_lonlat(initial_coords), zoom=3)
Map.add_basemap('HYBRID')

Map.clear_controls()
layer_control = LayersControl(position='topright')
Map.add_control(layer_control)
Map.add_control(ZoomControl(position='bottomright'))
draw_control = DrawControl(circle={}, polyline={}, rectangle={}, position='topleft')
Map.add_control(draw_control)

current_location_marker = CircleMarker(location=latlon_to_lonlat(initial_coords), name="location of interest")
Map.add_layer(current_location_marker)

vizParamsC = {
  'bands': ['threshold_' + str(initial_threshold)],
  'min': 0,
  'max': 1,
  'palette': ['000000', '909000'],
  'opacity': 0.5
}
vizParamsF = {
  'bands': ['threshold_' + str(initial_threshold)],
  'min': 0,
  'max': 1,
  'palette': ['000000', 'ff0000'],
  'opacity': 0.5
}
    
def update_location(geom):
    global current_geom
    global current_location_marker
    Map.remove_layer(current_location_marker)
    draw_control.clear()
    new_geom = geom
    if new_geom['type'] == 'Point':
        current_geom = ee.Geometry.Point(new_geom['coordinates'])
        ipl_loc = latlon_to_lonlat(new_geom['coordinates'])
        current_location_marker = CircleMarker(location=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
    else:
        current_geom = ee.Geometry.Polygon(new_geom['coordinates'])
        current_location_marker = Polygon(locations=latlon_to_lonlat(new_geom['coordinates']), name="location of interest")
        Map.add_layer(current_location_marker)
        
def drawcontrol_update(e):
    update_location(draw_control.last_draw['geometry'])

@debounce(0.5)
def uploadbutton_update(e):
    f = upload_button.data[-1]
    file_content = json.loads(codecs.decode(f, encoding="utf-8"))
    update_location(file_content['features'][0]['geometry'])
    polygon_coords = current_geom.coordinates().getInfo()[0]
    exes = np.array([i[0] for i in polygon_coords])
    whys = np.array([i[1] for i in polygon_coords])
    Map.setCenter(lat=np.mean(whys), lon=np.mean(exes))
    Map.set_trait('zoom', 9)


draw_control.observe(drawcontrol_update)
upload_button.observe(uploadbutton_update)

In [14]:
# STATUS
status_box = widgets.HTML()
def update_status(msg):
    html = '<div style="padding:20px; font-size:20pt"><br /><br /><br /><br />{}</div>'.format(str(msg))
    status_box.set_trait('value', html)

In [15]:
# TAB LAYOUT
map_box = widgets.VBox([upload_button, Map], layout=Layout(width='600px', height='600px'))
dashboard = widgets.HBox([map_box, status_box], layout=Layout(width='1600px', height='600px'))

In [16]:
dashboard

In [36]:
make_music()